# The National Bibliography of Scotland as Data
Created August-September 2020 for the National Library of Scotland's Data Foundry by Lucy Havens, Digital Library Research Intern

### About The National Bibliography of Scotland Dataset
This dataset is the first version of the bibliographic records for the National Bibliography of Scotland. This version of the National Bibliography of Scotland references materials published in Scotland and/or is in language Scots and/or is in language Scottish Gaelic from National Library of Scotland's main catalogue. This is the first iteration of the new National Bibliography of Scotland, which was originally produced in April 2019. National Bibliography of Scotland is an ongoing programme of work.
* Data format: metadata (as MARC XML)
* Data creation process: manual entry
* Data source: https://data.nls.uk/data/metadata-collections/national-bibliography-of-scotland/
***
### Table of Contents
0. [Preparation](#0.-Preparation)
1. [Data Cleaning and Standardisation](#1.-Data-Cleaning-and-Standardisation)
2. [Summary Statistics](#2.-Summary-Statistics)
3. [Exploratory Analysis](#3.-Exploratory-Analysis)
***

### 0. Preparation
Import libraries to use for cleaning, summarising and exploring the data:

In [7]:
# To prevent SSL certificate failure
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

# Libraries for data loading
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import string
import re
from collections import defaultdict

# Libraries for visualization
import altair as alt
import matplotlib.pyplot as plt

# Libraries for text analysis
# import nltk
# from nltk.tokenize import word_tokenize, sent_tokenize
# nltk.download('punkt')
# from nltk.corpus import PlaintextCorpusReader
# nltk.download('wordnet')
# from nltk.corpus import wordnet
# from nltk.corpus import stopwords
# from nltk.text import Text
# from nltk.stem.porter import PorterStemmer 
# from nltk.probability import FreqDist
# nltk.download('averaged_perceptron_tagger')
# from nltk.tag import pos_tag
# nltk.download('tagsets')  # part of speech tags
# from nltk.draw.dispersion import dispersion_plot as displt

Due to the large size of The National Bibliography of Scotland (NBS) data files, they aren't uploaded to the collections-as-data GitHub repo.  To load the NBS MARCXML data into this Notebook, please download the data from the [NLS Data Foundry website](https://data.nls.uk/data/metadata-collections/national-bibliography-of-scotland/).  Edit the file path below as necessary so that you can run this Notebook on your own computer.

The NBS data is actually **metadata**, meaning descriptive data about data.  In this case, the metadata contains information about books that have been published in Scotland, published in the language Scots, or published in the language Scottish Gaelic.  The metadata is provided as MARC XML.  MARC is a metadata standard used in libraries.  XML is a file format that is more widely used than MARC, so MARC is often provided as MARC *XML* so that systems other than library databases can read the data.

If you've never seen XML data before, check out [this sample](https://www.loc.gov/standards/marcxml/Sandburg/sandburg.xml) XML file of MARC metadata from the Library of Congress.  To learn more, I'd recommend starting with W3 Schools' [tutorial on XML](https://www.w3schools.com/xml/default.asp), which explains its purpose, structure (a tree), tag naming conventions, and much more.

To load the NBS MARC XML file, we'll use the Python library [ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html).  ElementTree, which we abbreviate *ET*, loads XML data (or metadata, in our case) in a hierarchical structure, or tree.  To iterate through the metadata, we need to find the root, or top-most level, of the tree.  From there we can travel up and down to pull out metadata of interest.

In [9]:
# Edit the file path in parentheses below to correspond to where the NBS MARC XML file that
# you downloaded can be found on your computer (unless the file is in a folder named Downloads!)
tree = ET.parse('data/National-Bibliography-of-Scotland-v1-dataset-MARC.xml')
root = tree.getroot()

Let's see how the metadata loaded:

In [22]:
print("Root tag:", str(root.tag))
# print("Root text:", str(root.text))               # empty
print("Root's first child tag:", str(root[0].tag))
print("Root's 4th grandchild tag:")
print("  ",root[0][4].tag)
print("     Datafield attribute:",root[0][4].attrib)
print("        Great grandchild tag:",root[0][4][0].tag)
print("        Great grandchild attribute:",root[0][4][0].attrib)
print("        Great grandchild text:",root[0][4][0].text)

Root tag: {http://www.loc.gov/MARC21/slim}collection
Root's first child tag: {http://www.loc.gov/MARC21/slim}record
Root's 4th grandchild tag:
   {http://www.loc.gov/MARC21/slim}datafield
     Datafield attribute: {'tag': '020', 'ind1': ' ', 'ind2': ' '}
        Great grandchild tag: {http://www.loc.gov/MARC21/slim}subfield
        Great grandchild attribute: {'code': 'a'}
        Great grandchild text: 1850980284


Knowing that the MARC field is a combination of the `tag` and `code` we can see that the MARC field we've printed is `020$a`, which is for an International Standard Book Number (ISBN).

**MARC** was first developed by the Library of Congress and has since been adopted by libraries around the world.  The NBS uses MARC Bibliographic, about which more can be read at [this website](https://www.loc.gov/marc/).  MARC contains hundreds of metadata fields that libraries can choose to use, with a certain number of required fields and many optional fields.  In MARC metadata, tags are indicated with 3-digit numbers, indicators are single-digit numbers, and subfields are indicated with lowercase letters. A space and pound sign (`#`) separates fields from indicators, and a dollar sign (`$`) separates the field from the subfield.  For example, the personal name (or primary author) metadata entry has:

* **tag:** ``100``
* **indicators:** ``0`` for forename and ``1`` for surname
* **subfields:** ``$a`` for personal name, ``$b`` for numeration, ``$c`` for titles and other words associated with a name, ``$q`` for a fuller form of the author's name, ``$d`` for dates associated with a name

A metadata entry for an author could look like:
```
100 1# $a Gregory, Ruth W.
       $q (Ruth Wilhelme),
       $d 1910-
```
Some MARC fields are repeatable, such as the *International Standard Book Number* (ISBN), while others are not, such as *Main entry -- Personal name* (author).  More detail and examples for commonly used fields are available [here](https://www.loc.gov/marc/umb/um07to10.html).

MARC**XML** use the same tags, indicators, and subfields as attributes inside tags `< >`.  For example, the MARC metadata entry above would be the following in MARCXML:
```
<datafield tag="100" ind1="1" ind2="">
    <subfield code="a">Gregory, Ruth W.</subfield>
    <subfield code="q">(Ruth Wilhelme)</subfield>
    <subfield code="d">1910-</subfield>
</datafield>
```

#### 0.1 Dataset Size
First, let's get a sense of how much metadata we have in the *National Bibliography of Scotland* (NBS) dataset:

In [11]:
records = root.getchildren()
total_records = len(records)
print("Total records so far:",total_records)

Total records so far: 368961


Note that I've print ``Total records so far`` rather than simply ``Total records``.  This is because the NBS is an in-progress work and we're using the first of what will be many future versions of the NBS.
Let's see what metadata has been documented for the first record (or first child of the root):

In [12]:
for child in records[0]:
    if 'datafield' in child.tag:
        print('Tag:', child.attrib['tag'])
        ind1 = re.match("\d", child.attrib['ind1'])
        ind2 = re.match("\d", child.attrib['ind2'])
        if ind1 != None:
            print('  Indicator 1:', ind1[0])
        if ind2 != None:
            print('  Indicator 2:', ind2[0])
            
        grandchildren = child.getchildren()
        for grandchild in grandchildren:
            print(' ', grandchild.attrib['code'], grandchild.text)

Tag: 020
  a 1850980284
Tag: 020
  a 9781850980285
Tag: 035
  a (StEdNL)2614-nlsdb-Voyager
Tag: 100
  Indicator 1: 1
  a Dryden, Derek.
Tag: 245
  Indicator 1: 1
  Indicator 2: 0
  a Salmon fishing on the South Esk Estuary :
  b Teachers' Booklet /
  c devised by Derek Dryden ; series editor Norman Nichol ; cover illustration by Archie Williams.
Tag: 260
  a Hamilton :
  b Hamilton College of Education,
  c 1978.
Tag: 300
  a 14 p. :
  b ill., maps ;
  c 30 cm.
Tag: 490
  Indicator 1: 1
  a Learning resources ;
  v B78
Tag: 700
  Indicator 1: 1
  a Nichol, Norman.
Tag: 700
  Indicator 1: 1
  a Williams, Archie.
Tag: 710
  Indicator 1: 2
  a Hamilton College of Education.
Tag: 830
  Indicator 2: 0
  a Learning resources ;
  v B78.
Tag: 919
  a NBS


Try replacing the number `0` with any number less than 368,961 to see how different records use different combinations of metadata fields!  (Remember that a list's maximum index is always 1 less than the length of a list, because the indeces begin at 0, not 1.)

#### 0.2 Identifying Subsets of the Data
Let's select a subset of MARC fields that we want to extract from the NBS MARCXML file we've loaded and put those selections in a **dataframe**.  Dataframes are essentially tables.  The Python library [Pandas](https://pandas.pydata.org/docs/), which we abbreviated `pd` at the start of this notebook, allows us to create dataframes and then run queries over their rows and columns to efficiently analyze the data.

**Step 1:** First, we'll create a dictionary (a data type in Python) that matches up MARC field with the name of the type of metadata the field contains (note that we're only defining a *subset* of all the available metadata field information):

In [13]:
marc_tags = ['100', '130', '245', '260', '650', '700', '710',]
marc_names = ['Author', 'Uniform title', 'Title statement', 
              'Publication, distribution, etc.', 'Subject added entry -- Topical term',
              'Added entry -- Personal name', 'Added entry -- Corporate name']

marc_dict = dict(zip(marc_tags,marc_names))
for key,value in marc_dict.items():
    print("MARC Tag:", key, "| Tag Name:", value)

MARC Tag: 100 | Tag Name: Author
MARC Tag: 130 | Tag Name: Uniform title
MARC Tag: 245 | Tag Name: Title statement
MARC Tag: 260 | Tag Name: Publication, distribution, etc.
MARC Tag: 650 | Tag Name: Subject added entry -- Topical term
MARC Tag: 700 | Tag Name: Added entry -- Personal name
MARC Tag: 710 | Tag Name: Added entry -- Corporate name


**Step 2:** We'll also create dictionaries for select subfields, one for each tag in the dictionary from step 1, and put them into a dictionary where each subfield dictionary (value) is associated with its corresponding tag (key):

In [14]:
author = {'a' : 'Personal name'}
unif_t = {'a' : 'Uniform title', 'l' : 'Language of work', 'f' : 'Date of work'}
title_stat = {'a' : 'Title proper'}
pub_dist = {'a' : 'Place', 'b' : 'Name', 'c' : 'Date'}
topic = {'a' : 'Topical term'}
pers_name = {'a' : 'Personal name', 'q' : 'Fuller name'}
corp_name = {'a' : 'Corporate or jurisdiction name'}

marc_subfields = [author, unif_t, title_stat, pub_dist, topic, pers_name, corp_name]
marc_tag_subfields = dict(zip(marc_tags, marc_subfields))
for key,value in marc_tag_subfields.items():
    print("MARC Tag:", key, "| Subfields:", value)

MARC Tag: 100 | Subfields: {'a': 'Personal name'}
MARC Tag: 130 | Subfields: {'a': 'Uniform title', 'l': 'Language of work', 'f': 'Date of work'}
MARC Tag: 245 | Subfields: {'a': 'Title proper'}
MARC Tag: 260 | Subfields: {'a': 'Place', 'b': 'Name', 'c': 'Date'}
MARC Tag: 650 | Subfields: {'a': 'Topical term'}
MARC Tag: 700 | Subfields: {'a': 'Personal name', 'q': 'Fuller name'}
MARC Tag: 710 | Subfields: {'a': 'Corporate or jurisdiction name'}


**Step 3:** Now let's extract the metadata field values from the NBS MARCXML metadata for the tags and subfields we defined in steps 1 and 2:

In [15]:
# To avoid rewriting similar code lines, we'll write a function in which we
# input a child element of the XML tree's root, a MARC tag, and a subfield of that tag,
# and receive as output the text of the MARC field if it's found (and False if it's not found)

def getSubfieldText(elem, marcTag, subfield):
    if (elem.attrib['tag'] == marcTag) :
            subelems = elem.getchildren()
            for subelem in subelems:
                if subelem.attrib['code'] == subfield:
                    return subelem.text
    else:
        return False

In [35]:
all_authors, all_titles, all_langs, all_pub_dates, all_pub_places, all_topics = [], [], [], [], [], []
for record in records:
    has_author = False
    has_title = False
    has_lang = False
    has_pub_date = False
    has_pub_place = False
    has_topic = False
    
    authors, titles, langs, pub_dates, pub_places, topics = [], [], [], [], [], []
    
    for child in record.findall('{http://www.loc.gov/MARC21/slim}datafield'):
        # Get author name in field 100$a
        author = getSubfieldText(child, "100", "a")
        if author:
            has_author = True
            authors += [author]

        # Get title in either field 130$a or 245$a
        title1 = getSubfieldText(child, "130", "a")
        title2 = getSubfieldText(child, "245", "a")
        if title2:
            has_title = True
            titles += [title2]
        elif title1:
            has_title = True
            titles += [title1]
            
        # Get language
        lang = getSubfieldText(child, "130", "l")
        if lang:
            has_lang = True
            langs += [lang]

        # Get publication date
        pub_date1 = getSubfieldText(child, "130", "f")
        pub_date2 = getSubfieldText(child, "260", "c")
        if pub_date2:
            has_pub_date = True
            pub_dates += [pub_date2]
        elif pub_date1:
            has_pub_date = True
            pub_dates += [pub_date1]

        # Get publication place
        pub_place = getSubfieldText(child, "260", "a")
        if pub_place:
            has_pub_place = True
            pub_places += [pub_place]

        # Get topical terms
        topic = getSubfieldText(child, "650", "a")
        if topic:
            has_topic = True
            topics += [topic]
        
    # After iterating through all datafield elements of the record
    # (the elements where MARC fields may be found), if a MARC field
    # we searched for isn't found, then add an empty string for that
    # record's MARC field text
    if not has_author:
        authors += ["None"]
    if not has_title:
        titles += ["None"]
    if not has_lang:
        langs += ["None"]
    if not has_pub_date:
        pub_dates += ["None"]
    if not has_pub_place:
        pub_places += ["None"]
    if not has_topic:
        topics += ["None"]

    all_authors.append(authors)
    all_titles.append(titles)
    all_langs.append(langs)
    all_pub_dates.append(pub_dates)
    all_pub_places.append(pub_places)
    all_topics.append(topics)
    
# There should be one sublist inside each all_xxx lists for every record (meaning they are all the same length)
# The assertions below will throw an error if this is not the case, indicating that our function didn't work as expected
assert len(all_topics) == len(all_titles)
assert len(all_titles) == len(all_authors)
assert len(all_authors) == len(all_langs)
assert len(all_langs) == len(all_pub_places)
assert len(all_pub_places) == len(all_pub_dates)
assert len(all_pub_dates) == len(records)

In [38]:
all_topics[0:10]

[['None'],
 ['Adult education'],
 ['Birds', 'Birds'],
 ['Adult education'],
 ['Nature conservation'],
 ['Wages.'],
 ['None'],
 ['Scientific expeditions'],
 ['None'],
 ['None']]

**Step 4:** Now we'll turn the lists we created of all topics, titles, authors, langauges, publication places, and publication dates into a dataframe, or table, using Pandas:

In [39]:
# First create a dictionary for each column that will be in the dataframe
cols = {'author' : all_authors, 'title' : all_titles, 'topic' : all_topics, 'language' : all_langs, 'publication_place' : all_pub_places, 'publication_date' : all_pub_dates}
df = pd.DataFrame(cols)
df.head()   # this prints the first 5 rows of a dataframe

,author,title,topic,language,publication_place,publication_date
0,"[Dryden, Derek.]",[Salmon fishing on the South Esk Estuary :],[None],[None],[Hamilton :],[1978.]
1,"[Macpherson, Iain.]",[Attracting new students to adult education :],[Adult education],[None],[Edinburgh :],[1989.]
2,[None],[The breeding birds of south-east Scotland :],"[Birds, Birds]",[None],[Edinburgh :],[1998.]
3,[None],"[Adult education, the challenge of change]",[Adult education],[None],[Edinburgh],[1975]
4,[None],[Nature conservation in the Cairngorms :],[Nature conservation],[None],[Edinburgh :],[[1989?].]


In [40]:
df.tail()   # df.tail prints the last 5 rows of a dataframe

,author,title,topic,language,publication_place,publication_date
368956,"[Lister, John,]","[Epigrams, and Jeux d'Esprit /]",[None],[None],[Edinburgh :],[1870.]
368957,[A. T. G.],[Border reminiscences. Annals of Thornlea [in ...,[None],[None],"[Galashiels, ]",[[1899]]
368958,[A. T. G.],"[""Lammermoor leaves"" /]",[None],[None],"[Galashiels,]",[1898.]
368959,[A. W. G.],[Dissolution of parliament. A statesman's adve...,[None],[None],"[Edinburgh, ]",[[1880]]
368960,[E. G.],[The Sabbath trader.],"[Sunday., Sabbath., Merchants]",[None],[Stirling :],[1855.]


### 1. Data Cleaning and Standardisation

[Code cells in this section will have one function each, preceded by comments as markdown above the cell to narrate the cleaning process]

In [ ]:
# code goes here

In [ ]:
# tokenisation

In [ ]:
# lemmatisation

In [ ]:
# stemming

In [ ]:
# part-of-speech tagging

### 2. Summary Statistics
[Code cells in this section will have one function each, preceded with comments in a markdown cell narrating the summarisation process]

#### 2.1 Frequencies and Sizes

[Narration]

In [ ]:
# code goes here

#### 2.2 Uniqueness and Variety

[Narration]

In [ ]:
# code goes here - most frequent words, sentence structure

### 3. Exploratory Analysis (this section will be included for 2-3 datasets)
[Code cells in this section will have one function each, preceded with comments in a markdown cell posing an exploratory research question]

#### 3.1 [exploratory research question 1]

In [ ]:
# code goes here

In [ ]:
# visualizations go here

#### 3.2 [exploratory research question 2]

In [ ]:
# code goes here

In [ ]:
# visualizations go here